In [7]:
import requests
import pandas as pd
import sqlite3
import logging

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Função para criar a tabela no banco de dados SQLite (se ainda não existir)
def criar_tabela():
    with sqlite3.connect('dados_pontos.db') as conn:
        conn.execute('''
            CREATE TABLE IF NOT EXISTS onibus (
                NOME TEXT,
                NUM TEXT,
                Latitude REAL,
                Longitude REAL,
                SEQ INTEGER,
                GRUPO TEXT,
                SENTIDO TEXT,
                TIPO TEXT,
                ID_do_Itinerario INTEGER,
                COD TEXT
            )
        ''')
        conn.commit()

# Função para buscar e processar dados da linha de ônibus
def buscar_e_processar_dados(linha):
    url = f'https://transporteservico.urbs.curitiba.pr.gov.br/getPontosLinha.php?linha={linha:03}&c=821f0'
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        dados_json = response.json()

        if not dados_json:
            return None

        # Criar DataFrame a partir dos dados
        df = pd.DataFrame.from_records(dados_json)

        # Substituir vírgulas por pontos e converter tipos
        df['Latitude'] = df['LAT'].str.replace(',', '.').astype(float)
        df['Longitude'] = df['LON'].str.replace(',', '.').astype(float)
        df['SEQ'] = df['SEQ'].astype(int)
        df['ID_do_Itinerario'] = df['ITINERARY_ID'].astype(int)
        
        # Adicionar coluna COD
        df['COD'] = linha

        return df[['NOME', 'NUM', 'Latitude', 'Longitude', 'SEQ', 'GRUPO', 'SENTIDO', 'TIPO', 'ID_do_Itinerario', 'COD']]

    except requests.RequestException as e:
        logging.error(f"Erro na requisição para a linha {linha}: {e}")
    except Exception as e:
        logging.error(f"Erro ao processar a linha {linha}: {e}")
    return None

# Função para carregar a tabela de ônibus em um DataFrame
def carregar_tabela_para_dataframe(nome_tabela, db='dados_pontos.db'):
    try:
        with sqlite3.connect(db) as conn:
            cursor = conn.cursor()

            cursor.execute("""
                SELECT name FROM sqlite_master WHERE type='table' AND name=?;
            """, (nome_tabela,))
            resultado = cursor.fetchone()

            if resultado:
                df = pd.read_sql(f'SELECT * FROM {nome_tabela}', conn)
            else:
                df = pd.DataFrame()

        return df
    except Exception as e:
        logging.error(f"Erro ao carregar a tabela {nome_tabela}: {e}")
        return pd.DataFrame()

# Função para gravar os dados no banco de dados SQLite
def gravar_no_banco(df_result):
    try:
        with sqlite3.connect('dados_pontos.db') as conn:
            df_result.to_sql('onibus', conn, if_exists='append', index=False)
    except Exception as e:
        logging.error(f"Erro ao gravar no banco de dados: {e}")

# Função para processar as linhas de ônibus de forma sequencial
def processar_linhas_sequencialmente(linhas_unicas, df_onibus):
    result_list = []

    for linha in linhas_unicas:
        df_result = buscar_e_processar_dados(linha)
        if df_result is not None:
            result_list.append(df_result)

    if result_list:
        df_concatenado = pd.concat(result_list, ignore_index=True)
        
        df_onibus['COD'] = df_onibus['COD'].astype(int)
        df_novos_pontos = df_concatenado[~df_concatenado['COD'].isin(df_onibus['COD'])]

        if not df_novos_pontos.empty:
            if not df_onibus.empty:
                gravar_no_banco(df_novos_pontos)
                logging.info(f"Novas linhas encontradas, total de linhas inseridas: {len(df_novos_pontos)}")
                logging.info(f"Novas linhas: {df_novos_pontos['COD'].unique()}")
            else:
                gravar_no_banco(df_concatenado)
                logging.info(f"Base de dados vazia, total de linhas inseridas: {len(df_concatenado)}")
                logging.info(f"Novas linhas: {df_novos_pontos['COD'].unique()}")
        else:
            logging.info("Nenhum novo ponto para gravar.")

# Função para remover duplicatas
def remover_duplicatas():
    logging.info("Removendo duplicatas da tabela onibus...")
    
    with sqlite3.connect('dados_pontos.db') as conn:
        cursor = conn.cursor()
        
        cursor.execute('''
            DELETE FROM onibus
            WHERE rowid NOT IN (
                SELECT MIN(rowid)
                FROM onibus
                GROUP BY COD, Latitude, Longitude
            )
        ''')
        conn.commit()

# Inicializar o processamento sequencial
def iniciar_processamento():
    linhas_unicas = range(0, 1000)  # Exemplo de intervalo de linhas
    criar_tabela()  # Garantir que a tabela está criada

    try:
        df_onibus = carregar_tabela_para_dataframe('onibus')
        processar_linhas_sequencialmente(linhas_unicas, df_onibus)
        remover_duplicatas()  # Chamar a função de remoção de duplicatas
    except Exception as e:
        logging.error(f"Erro durante o processamento: {e}")
    finally:
        logging.info("Processamento finalizado.")

iniciar_processamento()


2024-10-21 12:23:21,878 - ERROR - Erro na requisição para a linha 450: HTTPSConnectionPool(host='transporteservico.urbs.curitiba.pr.gov.br', port=443): Read timed out. (read timeout=10)
2024-10-21 12:24:21,940 - ERROR - Erro na requisição para a linha 671: HTTPSConnectionPool(host='transporteservico.urbs.curitiba.pr.gov.br', port=443): Read timed out. (read timeout=10)
2024-10-21 12:24:35,889 - ERROR - Erro na requisição para a linha 687: HTTPSConnectionPool(host='transporteservico.urbs.curitiba.pr.gov.br', port=443): Read timed out. (read timeout=10)
2024-10-21 12:25:01,088 - ERROR - Erro na requisição para a linha 761: HTTPSConnectionPool(host='transporteservico.urbs.curitiba.pr.gov.br', port=443): Read timed out. (read timeout=10)
2024-10-21 12:25:25,444 - ERROR - Erro na requisição para a linha 828: HTTPSConnectionPool(host='transporteservico.urbs.curitiba.pr.gov.br', port=443): Read timed out. (read timeout=10)
2024-10-21 12:26:00,208 - INFO - Nenhum novo ponto para gravar.
2024-1

In [6]:
df = carregar_tabela_para_dataframe('onibus')
df

,NOME,NUM,Latitude,Longitude,SEQ,GRUPO,SENTIDO,TIPO,ID_do_Itinerario,COD
0,"Av. Manoel Ribas, 531 - Mercês",110037,-25.422483,-49.284328,37,,Horário,Novo mobiliário,481,10
1,"Rua Augusto Stresser, 517 - Juvevê",130238,-25.417083,-49.256173,6,,Horário,Novo mobiliário,481,10
2,"Rua Chile, 821 - Rebouças",140051,-25.447454,-49.257707,18,,Horário,Novo mobiliário,481,10
3,"R. Dep. Mário de Barros, 1681 - Centro Cívico",120112,-25.411726,-49.270902,42,,Horário,Novo mobiliário,481,10
4,"R. Alm. Tamandaré, 272 - Alto da Rua Quinze",130268,-25.430345,-49.255576,10,,Horário,Novo mobiliário,481,10
...,...,...,...,...,...,...,...,...,...,...
17540,"R. Des. Hugo Simas, 2300 - Bom Retiro",120333,-25.400987,-49.285160,12,,Nestor de Castro,Novo mobiliário,8654,171
17541,"Rua Des. Benvindo Valente, 104 - São Francisco",110011,-25.420492,-49.275230,4,,Primavera,Novo mobiliário,8655,171
17542,"R. Amauri Lange Silvério, 232 - Pilarzinho",120032,-25.395021,-49.285215,15,,Primavera,Novo mobiliário,8655,171
17543,"R. Des. Hugo Simas, 2672 - Pilarzinho",120024,-25.397184,-49.287881,14,,Primavera,Novo mobiliário,8655,171
